In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [9]:
#load data
data = pd.read_csv(r"C:\Users\migui\OneDrive - UFV\4ºBA\TFG\SetDatos\dota2_chat_messages.csv")

In [10]:
print(data.head())  # Muestra las primeras filas
print(data.info())  # Información de columnas y tipos de datos
print(data.describe())  # Estadísticas de las columnas numéricas


   match        time  slot            text
0      0  1005.12122     9       ладно гг 
1      0  1005.85442     9             изи
2      0  1008.65372     9              од
3      0  1010.51992     9          ебаный
4      0  1013.91912     9  мусор на войде
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21659448 entries, 0 to 21659447
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   match   int64  
 1   time    float64
 2   slot    int64  
 3   text    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 661.0+ MB
None
              match          time          slot
count  2.165945e+07  2.165945e+07  2.165945e+07
mean   4.996186e+05  1.457327e+03  4.504226e+00
std    2.888392e+05  9.540682e+02  2.875228e+00
min    0.000000e+00 -1.068088e+03  0.000000e+00
25%    2.493490e+05  7.288557e+02  2.000000e+00
50%    4.994910e+05  1.501327e+03  5.000000e+00
75%    7.498670e+05  2.118162e+03  7.000000e+00
max    9.999990e+05  8.443546e+03  9.000000e+00
